In [1]:
#Importing required modules
import pandas as pd
import numpy as np


This is a program used to prepare data for analysis - especially gender selection based on first name as well as adding a year based on date and some general clean-up. The data can then be further analysed, for example with Power BI. 

The datasets themselves are confidential, so all display has been blanked out.

In [2]:
#Reading the name-gender data and creating an appropriate dictionary
genderData = pd.read_excel("name_gender.xlsx")
gender_dict = dict(zip(genderData.FirstName, genderData.Gender))

In [3]:
#Reading the data
shelterData = pd.read_excel("Constituents_Donations_Cleaned_joined.xlsx")
#shelterData.head(20)

In [4]:
#shelterData.columns


In [5]:
#Some Prep Work: 
#Set all Gender to 'Unknown' to avoid empty fields, also set all Gender to 'N/A' if the Org_Flag is an Organization
for index, row in shelterData.iterrows():
    gender = row["Gender"]
    org = row["Org_Flag"]
    if org == 'Organization':
        shelterData.at[index, "Gender"] = "NotApp"
    elif pd.isna(gender):
        shelterData.at[index, "Gender"] = "Unknown"
        #Replace all NaN with 'Private' in Org_Name (for future processing)
    if pd.isna(row["Org_Name"]):
        shelterData.at[index, "Org_Name"] = "Private"
   

print(shelterData['Gender'].unique())

['Unknown' 'Male' 'Female' 'NotApp']


In [6]:
#First run through for determining the gender
for index, row in shelterData.iterrows():
    name=row["First_Name"]
    if row["Gender"] == "Unknown":
        if name in gender_dict:
            shelterData.at[index, "Gender"] = gender_dict[name]
            
#shelterData.head()


In [7]:
#Analyzing 'remnants'

#Just going by first name to exclude double names
for index, row in shelterData.iterrows():
    if row["Gender"]  == 'Unknown':
        splitName = row["First_Name"].rsplit(" ")
        
        if splitName[0] == "Estate":
            shelterData.at[index, "Org_Name"] = "Estate"
        
        #Just loop through each name and see if any string with a length> 1 fits 
        
        i = 0
        while i < len(splitName):
            if len(splitName[i])>1:
                if splitName[i] in gender_dict:
                    shelterData.at[index, "Gender"] = gender_dict[splitName[i]]
                    break
                #Also look at any split names like Jo-Ann
                elif '-' in splitName[i]:
                    nameCheck = splitName[i].rsplit('-')
                    j = 0
                    while j < len(nameCheck):
                        if len(nameCheck[j])>1:
                            if nameCheck[j] in gender_dict:
                                shelterData.at[index, "Gender"] = gender_dict[nameCheck[j]]
                                break
                        j +=1
                    
            i += 1
             
#shelterData.head(20)

In [8]:
#print(shelterData.dtypes)

In [9]:
#Creating an FSA field for postal code

#Need to create a list of FSA Codes first - this is a bit overkill, Nellie's shelter is toronto-centric, but
#I want to use this list at a later date for other projects.
#Starting at the second elment - the first name is not an FSA (it is 'Canada')

fsaData = pd.read_csv("Geo_starting_row_CSV.csv")

fsa=[]

for index, row in fsaData.iterrows():
    if index > 0:
        fsa.append(row['Geo Name'])

  
    


In [10]:
#Now retrieve the first three elements of the postal code. If they are an FSA, add this FSA in a special column. If not an FSA,
#add 'N/A', as this may be needed for later processing.

fsaList = []

for index, row in shelterData.iterrows():
    postalCode = str(row['Postal_Code'])[:3]
    if postalCode in fsa:
        fsaList.append(postalCode)
    else:
        fsaList.append('NotApp')

#print(fsaList)
    
shelterData['FSA'] = fsaList

In [11]:
#Simplification

shelterData = shelterData.drop(['Gift_Amount', 'Appeal_Description', 'Title'], axis=1)

#nelliesData.head()

In [12]:

#Creating a 'year' and 'month' field

shelterData['year'] = pd.DatetimeIndex(shelterData['Gift_Date']).year
shelterData['month'] = pd.DatetimeIndex(shelterData['Gift_Date']).month
    
#shelterData.head()        

In [13]:
shelterData.to_csv("UpdatedShelterData.csv")